<a href="https://colab.research.google.com/github/KevinBarranco24/Practica3_metrics/blob/main/Practica_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Práctica 3
##Métricas

###Kevin Kristop Morales Barranco - 214603492

#MODULES

In [2]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from scipy.stats import wrapcauchy

from scipy.spatial import distance

#CLASSES

In [3]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

#FUNCTIONS

##Brownian Motion (BM)


In [4]:
def bm_2d(n_steps=1000, speed=6, s_pos=[0,0]):
  #n_steps = 1000
  #s_pos = [0,0]
  #speed = 6

  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])

  temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    #turn_angle = np.random.choice([0, np.pi/2, np.pi, 3*np.pi/2])
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x, 'y_pos':BM_2d_df.y_pos[i]+velocity.y}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    return BM_2d_df

In [7]:
BM_2d_df = bm_2d(2000, 3, [5,7])

# Init figure
fig_BM_2d = go.Figure()

# Plot trajectory
fig_BM_2d.add_trace(go.Scatter(
    x = BM_2d_df.x_pos,
    y = BM_2d_df.y_pos,
    marker = dict(size=2),
    line = dict(width=1),
    mode='lines',
    name='BM_2d_trajectory',
    showlegend=True
))

fig_BM_2d.show()

#Correlated Random Walks (CRW)

In [43]:
def CRW(CRW_exponents=0.2, resolution=800, speed=6, s_pos=[0,0]):
  #init variables

  #resolution = 1000
  #s_pos = [0,0]
  #speed = 6

  #init arrays
  aux_domain = np.linspace(0, 2*np.pi, resolution)

  #init velocity vector
  velocity = Vec2d(speed, 0)

  wrapcauchy_rvs = wrapcauchy.rvs(CRW_exponents, size=resolution)


  #init DataFrames
  CRW_2d_df_s = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  CRW_2d_df_s = pd.concat([CRW_2d_df_s,temp_df], ignore_index=True)

  for j in range(resolution):
    velocity = velocity.rotated(wrapcauchy_rvs[j])

    temp_df = pd.DataFrame([{'x_pos':CRW_2d_df_s.x_pos[j]+velocity.x,'y_pos':CRW_2d_df_s.y_pos[j]+velocity.y}])
    CRW_2d_df_s = pd.concat([CRW_2d_df_s,temp_df], ignore_index=True)


  return CRW_2d_df_s
  #print(CRW_2d_df)


In [59]:
#print(CRW(0.5,16,3,[0,0]))
BM_2d_df = CRW(1.2,1,5,[5,7])

# Init figure
fig_BM_2d = go.Figure()

# Plot trajectory
fig_BM_2d.add_trace(go.Scatter(
    x = BM_2d_df.x_pos,
    y = BM_2d_df.y_pos,
    marker = dict(size=2),
    line = dict(width=1),
    mode='lines',
    name='CRW_trajectory',
    showlegend=True
))

fig_BM_2d.show()
